# Llama 2 - Setup

Llama ist ein offenes Sprachmodell, welches von Meta entwickelt und trainiert wurde.
Auf Basis des Vorgängers, der wohl unabsichtlicht im Internet aufgetaucht ist, wurden nicht nur von Wissenschaffenden leistungsfähige Derivate entwickelt.
Der GitHub-Nutzer [oobabooga](https://github.com/oobabooga) hat durch seine webbasierte Nutzungsschnittstelle namens [text-generation-webui](https://github.com/oobabooga/text-generation-webui) die Einrichtung und die Bereitstellung von LLama-Modellen stark vereinfacht. Ein weiteres populares Projekt [GPT4All](https://gpt4all.io/index.html).

### Konfiguration

In dem hier vorgestellten Arbeitsablauf, muss nur der Installationsort festgelegt werden:

In [ ]:
from pathlib import Path

# Where should the webui be downloaded to?
LLAMA_ROOT = Path("../text_gen").resolve()

### Plattformspezifische Anpassungen

Im besten Fall werden die restlichen Parameter automatisch gesetzt. Sollte das jedoch nicht funktionieren, können Nutzende die Umgebungsvariablen für den folgenden Installer auch händisch setzen.

In [ ]:
import platform
import subprocess

if platform.system() == "Darwin":
    OS = "macos"
elif platform.system() == "Windows":
    OS = "windows"
else:
    OS = "linux"

INSTALLER = LLAMA_ROOT.joinpath(f"start_{OS}.{'bat' if OS == 'windows' else 'sh'}")

try:
    subprocess.check_output("nvidia-smi")
    GPU_CHOICE = "A"  # use CUDA
except Exception:
    if OS == "macos" and platform.machine() == "arm64":
        GPU_CHOICE = "C"  # use MPS for M generation Macs
    else:
        GPU_CHOICE = "N"  # use CPU only

### Text Generation Web UI herunterladen

Es ist wieder Zeit ein Projekt zu 'klonen'!

In [ ]:
!git clone https://github.com/oobabooga/text-generation-webui.git {LLAMA_ROOT}

### Text Generation Web UI starten

Wir nutzen den 'One-Click-Installer', den `oobabooga`.
Dieser lädt `conda` zwar erneut herunter, was aber in Anbetracht der Größe der herunterzuladenden Modelle nicht weiter ins Gewicht fällt.

In [ ]:
%env GPU_CHOICE {GPU_CHOICE}
# If you know you have an AMD GPU and are on Linux or MacOS, you can set 
# GPU_CHOICE manually to "B" or "D" if you got an Intel Arc (IPEX) GPU
# %env GPU_CHOICE = B
%env USE_CUDA118 N  # if you have issues set this one to "Y"
# This is only needed for remote hosting
# %env COMMANDLINE_ARGS --share --gradio-auth {GRADIO_USER}:{GRADIO_PASS}

!{INSTALLER}

### Wir brauchen Modelle

<div style="text-align: center; margin-top: 1rem;">
   <img src="images/huggingface.svg" width="80"><span style="margin-left: 1rem; font-size: 4rem; font-weight:400; vertical-align: bottom;">Hugging Face</span>
</div>

> Hugging Face, the most popular AI tool in Data Science, has been an attention magnet for 316.6 million traffic between September 2022 and August 2023. -- writerbuddy.ai [¹](https://writerbuddy.ai/blog/ai-industry-analysis)

Die wahrscheinlich aktuell größte Community mit der größten Sammlung an GAN-Modellen findet man auf [huggingface.co](https://huggingface.co). Die Seite wird vom US-amerikanischen Unternehmen [Hugging Face, Inc](https://en.wikipedia.org/wiki/Hugging_Face) betreut.

Die Web UI erlaubt es, Sprachmodelle direkt von [Hugging Face](https://huggingface.co) (HF) herunterzuladen.
Dazu muss im Reiter `Model` nur der sogenannte HF-Handle eingegeben werden.
Sollte der Handle nicht eindeutig sein, kann noch die spezifische Modeldatei angegeben werden.

Die Auswahl an Sprachmodellen ist unüberschauber und neu trainierte Modelle werden täglich veröffentlicht.
Eine Orientierung bietet das [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard) von Hugging Face, welches Sprachmodelle und deren Leistungsfähigkeit in Benchark-Tests gegenüberstellt.

### Eine Modellauswahl

* [Intel/neural-chat-7b-v3-1](https://huggingface.co/Intel/neural-chat-7b-v3-1)
  + aktuell *im Schnitt* eines der besten 'generischen' Modelle
* [LeoLM/leo-hessianai-13b](https://huggingface.co/LeoLM/leo-hessianai-13b)
  + speziell für die deutsche Sprache optimiert
* [TheBloke/leo-hessianai-7B-GGUF](https://huggingface.co/TheBloke/leo-hessianai-7B-GGUF)
  + File: leo-hessianai-7b.Q5_K_M.gguf
  + Alternative: [TheBloke/leo-hessianai-7B-GPTQ:gptq-8bit-32g-actorder_True](https://huggingface.co/TheBloke/leo-hessianai-7B-GPTQ)
  + quantisiertes Version von LeoLM
* [TheBloke/openinstruct-mistral-7B-GPTQ:gptq-8bit-32g-actorder_True](https://huggingface.co/TheBloke/openinstruct-mistral-7B-GPTQ)
  + aktuell *im Schnitt* eines der besten Modelle speziell für Anweisungen trainiert
  + quantisierte Version

### Anmerkungen zu dem Namensschema von Modellen

Viele Modelle gibt es nicht nur in einer Variante. Um es einfacher zu machen, um welche Variante es sich handelt, gibt es einige Namenszusätze: `13B`/`7B`/`30B` gibt die Anzahl der Parameter des Modells an. Mehr bedeutet grob gesprochen genauere Ergebnisse, aber auch komplexere Berechnungen und einen größeren Speicherbedarf.

Neben der Reduktion der Parameter, ist ein weiterer Optimierungsschritt die Quantisierung.
Hierbei werden Parameter und Gewichte in Zahlenformate überführt die weniger Speicher benötigen.
Dadurch wird die Genauigkeit der Berechnungen reduzierert, was jedoch bis zu einem gewissen Grad die Ausgabe der Modelle nur marginal negativ beeinflusst: `GGML` und `GGUF` geben bspw. Quantisierungsmethode an, wobei `GGUF` nach und nach `GGML` ersetzt. Einige Bibliotheken haben die Unterstützung von `GGML` bereits eingestellt. Die Art der für Quantifizierungsmethode und die Anzahl der genutzen Bits bei `GGUF`/`GGML` wird teils als weitere Zusätze. Bspw. steht `Q5_K_M` für 5 genutzte Bits und `GGML_TYPE_Q6_K`-Quantizierung [¹](https://github.com/ggerganov/llama.cpp/pull/1684)). Weitere Quantisierungs- und Komprimierungsmethoden `AWQ` und `GPTQ`, die teils mit einem stärken Fokus auf GPU-Berechnungen weitere Effizienzgewinne bedeuten können.

Andere Zusätze wie `chat` oder `instruct` implizieren, dass die Modelle speziell für Konversationsszenarien oder zur Ausgabe bzw. zum Befolgen von Instruktionen trainiert/feinjustiert worden sind.